# Đề tài: Exploratory Data Analysis of Cryptocurrency Historical Data

Sinh viên thực hiện:

- Lê Thanh Hiếu - 1712434

- Phạm Minh Thắng - 1712759  

Link colab: https://colab.research.google.com/drive/1TB8TWzztma41wrzVkWFVsbnkxZZkHKzd?usp=sharing

Notebook này dành cho phần thu thập dữ liệu

## Data Crawling

### Cài đặt các công cụ cần thiết

Cài đặt Selenium và Chromedriver phục vụ cho việc thu thập dữ liệu

In [ ]:
import pandas as pd

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)

### Thu thập dữ liệu

Toàn bộ dữ liệu được thu thập từ trang [Coinmarketcap](https://coinmarketcap.com). 

Về [Coinmarketcap](https://coinmarketcap.com), đây là trang web theo dõi giá tiền điện tử được tham khảo nhiều nhất trên thế giới, trong thời đại mà tiền điện tử ngày càng phát triển nhanh chóng. [Coinmarketcap](https://coinmarketcap.com) cung cấp thông tin chính xác, hiệu quả và trực quan về tiền điện tử cho người sử dụng.

Trang chủ của [Coinmarketcap](https://coinmarketcap.com) hiển thị danh sách các loại tiền điện tử hiện có trên thị trường, được sắp xếp theo giá trị vốn hoá thị trường. Nhóm tiến hành crawl dữ liệu danh sách các đồng tiền trước tiên.

Việc thu thập dữ liệu được thực hiện bằng cách parse HTML với thư viện Selenium (do trang web có sử dụng JavaScript với cơ chế lazy loading).

Các thông tin thu thập được từ danh sách bao gồm:

 - Tên đầy đủ của coin.
 - Ký hiệu của coin.
 - Giá trị của coin tại thời điểm thu thập dữ liệu.
 - Sự thay đổi giá so với 24 giờ/7 ngày trước.
 - Giá trị vốn hoá thị trường (USD).
 - Khối lượng giao dịch trong vòng 24h (theo USD và theo loại coin tương ứng).
 - Tổng lượng coin đang được lưu thông trên thị trường.
 - URL dẫn đến trang cung cấp thông tin chi tiết từng loại coin.


In [ ]:
home_url = "https://coinmarketcap.com"
driver.get(home_url)

In [ ]:
driver.find_element_by_class_name("hKnmWV").click()
driver.find_element_by_class_name("jvQpLZ").find_element_by_xpath("button[3]").click()
driver.find_element_by_class_name("cmc-cookie-policy-banner__close").click()
driver.find_element_by_tag_name("html").send_keys(Keys.END)

In [ ]:
coin_file = open("data/coin_list.csv", "w+")
coin_file.write("name\tsymbol\tprice\t24h\t7d\tmarket_cap\tvolume(usd)\tvolume(coin)\tcirculating_supply\turl\n")

In [ ]:
while True:
    print("Page:", driver.find_element_by_class_name("rc-pagination-item-active").get_attribute("title"))
    lines = driver.find_element_by_class_name("tableWrapper___3utdq").find_element_by_tag_name("tbody").find_elements_by_tag_name("tr")
    for line in lines:
        try:
            content = line.text.replace("\n", "\t")
            link = line.find_element_by_tag_name("a").get_attribute("href") + "historical-data/?start=20100101&end=20201215"
            coin_file.write(f"{content}\t{link}\n")
        except:
            pass

    next = driver.find_element_by_class_name("eEiCJF").find_element_by_class_name("rc-pagination-next")
    if "rc-pagination-disabled" not in next.get_attribute("class"):
        next.click()
        time.sleep(1)
    else:
        break

coin_file.close()

Sau khi đã có được danh sách các đồng tiền điện tử, nhóm tiến hành thu thập lịch sử thay đổi giá của các đồng tiền trên. Để thu thập lịch sử giá từng loại coin, nhóm sử dụng URL được thu thập trong danh sách các loại coin. Lịch sử được thu thập từ ngày đầu tiên coin có mặt trên thị trường đến ngày 16/12/2020.

In [ ]:
coin_list = pd.read_csv("data/coin_list.csv", sep="\t")

In [ ]:
coin_list.dropna(subset=["url"], inplace=True)
coin_list.drop_duplicates(subset=["name", "symbol"],inplace=True)
coin_list.index = coin_list.index.astype(int)

In [ ]:
coin_list

,name,symbol,price,24h,7d,market_cap,volume(usd),volume(coin),circulating_supply,url
1,Bitcoin,BTC,"$26,726.84",3.61%,17.11%,"$497,741,034,997","$63,368,452,500","2,365,883 BTC","18,583,331 BTC",https://coinmarketcap.com/currencies/bitcoin/h...
2,Ethereum,ETH,$727.18,12.94%,18.28%,"$83,066,027,790","$33,108,494,005","45,444,903 ETH","114,016,892 ETH",https://coinmarketcap.com/currencies/ethereum/...
3,Tether,USDT,$0.998099,0.03%,0.06%,"$20,781,363,466","$98,199,951,695","98,386,620,807 USDT","20,820,866,934 USDT",https://coinmarketcap.com/currencies/tether/hi...
4,XRP,XRP,$0.296772,2.27%,44.09%,"$13,453,037,560","$9,457,140,954","31,917,869,612 XRP","45,404,028,640 XRP",https://coinmarketcap.com/currencies/xrp/histo...
5,Litecoin,LTC,$134.63,0.99%,27.06%,"$8,993,704,276","$13,904,633,547","102,328,515 LTC","66,187,462 LTC",https://coinmarketcap.com/currencies/litecoin/...
...,...,...,...,...,...,...,...,...,...,...
4090,WELL,WELL,$0.302254,3.31%,12.78%,$0,$0,0 WELL,0 WELL,https://coinmarketcap.com/currencies/well-toke...
4096,Alpha5,A5T,$0.666347,0.07%,1.57%,$0,$0,0 A5T,0 A5T,https://coinmarketcap.com/currencies/alpha5/hi...
4097,Sugarchain,SUGAR,$0.003149,1.71%,0.82%,$0,$0,0 SUGAR,0 SUGAR,https://coinmarketcap.com/currencies/sugarchai...
4111,Decentralized Bank,DEBA,$0.026548,13.92%,2.24%,$0,$0,0 DEBA,0 DEBA,https://coinmarketcap.com/currencies/decentral...


In [ ]:
for i in range(len(coin_list)):
    coin = coin_list.iloc[i]
    symbol = coin["symbol"]
    f = open(f"data/{symbol}.csv", "w+")
    f.write("date\topen\thigh\tlow\tclose\tvolume\tmarket_cap\n")

    url = coin["url"]
    driver.get(url)

    time.sleep(10)

    try:
        data_table = driver.find_element_by_tag_name("tbody")
    except:
        continue

    content = data_table.text.split("\n")
    count = 0
    for value in content:
        f.write(value)
        count += 1
        if count < 7:
            f.write("\t")
        else:
            f.write("\n")
            count = 0
    f.close()
        